# Fog Density Estimation with Convolutional Neural Networks

In this notebook I use a convolutional neural network (CNN) for fog density classification from CCTV images. The network is trained on images obtained from KNMI weather stations in De Bilt and Cabauw. 

Neural networks that are trained from scratch need a lot of data to allow itself to obtain decent weights. For that reason, pre-trained CNN can be very useful. These networks are trained on ImageNet, an image dataset containing 1000 generic classes. The network can then be 'fine-tuned' on the weather data. For this notebook, I will use the InceptionV3 CNN with pre-trained ImageNet weights.

In [8]:
from keras.layers import Dropout, Dense, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.models import Model
from keras.optimizers import SGD
from keras import backend as K
from keras.applications import InceptionV3
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import cv2
import glob
import os
K.set_image_dim_ordering('tf')

## Processing

In [6]:
''' Specify the paths to the data.
'''
train_dir = '../data/Training/'

''' Get image metadata
'''
meta = pd.read_csv(train_dir + 'ImageDescription2.csv')
imgID = meta['image_id'].values
y_meta = meta['vis_class'].factorize()
y = list(y_meta[0])
basenames = meta['basename'].values

''' Define functions to load the training images.
'''
def process_img(path):
    return cv2.resize(cv2.imread(path), (img_width, img_height), interpolation=cv2.INTER_LINEAR)

def load_data(data_dir, y):
    X = [process_img(glob.glob(os.path.join(data_dir, basename))[0]) for basename in basenames]
    X = np.array(X, dtype=np.uint8).transpose((0,1,2,3)).astype('float32') / 255
    y = np_utils.to_categorical(np.array(y, dtype=np.uint8), 4)
    return X, y

''' Load images tailored for network input. 
However, input dimensions are required to be at 
least 224x224x3.
'''
img_width = img_height = 224
X, y = load_data(train_dir, y)


''' Create stratified train test split
'''
X_cab, y_cab = X[2517:], y[2517:]
X_bil, y_bil = X[:2517], y[:2517]

X_trainc, X_testc, y_trainc, y_testc = train_test_split(X_cab, y_cab, test_size=0.2)
X_trainb, X_testb, y_trainb, y_testb = train_test_split(X_bil, y_bil, test_size=0.2)

X_train = np.append(X_trainc, X_trainb, axis=0)
X_test = np.append(X_testc, X_testb, axis=0)
y_train = np.append(y_trainc, y_trainb, axis=0)
y_test = np.append(y_testc, y_testb, axis=0)

del X_trainc, X_trainb, X_testc, X_testb, y_trainc, y_trainb, y_testc, y_testb

## Create model

In [13]:
'''
Load the network. Pre-trained Imagenet weights are loaded into the models.
'''
print('Loading network with pre-trained weights...')
base = InceptionV3(weights='imagenet', include_top=False)


'''
Define a custom fully connected layer to replace the top layers. This is necessary to
support 4 output classes, instead of the default 1000.
'''
x = base.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
preds = Dense(4, activation='softmax')(x)

model = Model(inputs=base.input, outputs=preds)

'''
Because we append an untrained fully connected layer, we have to train it
exclusively, i.e.: freeze the weights of the network. Otherwise, the randomly 
initialized weights of our fully connected layer causes random error to 
back-propagate into the 'correct' weights.
'''
for layer in base.layers:
    layer.trainable = False


# Compile the neural network. 
model.compile(optimizer=SGD(lr=1e-3, decay=1e-6, momentum=.9, nesterov=True),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

'''
Train the fully connected layer. Earlier runs have shown that 2 epochs
are required before the network starts over-fitting.
'''
print('Fully connected layer training initialized...')
model.fit(x=X_train, y=y_train, validation_split=0.2, epochs=2, batch_size=50)

Loading network with pre-trained weights...
Fully connected layer training initialized...
Train on 3064 samples, validate on 766 samples
Epoch 1/2
3064/3064 [==============================] - 507s - loss: 1.3401 - acc: 0.3858 - val_loss: 1.1852 - val_acc: 0.4465
Epoch 2/2
3064/3064 [==============================] - 495s - loss: 1.1938 - acc: 0.4634 - val_loss: 1.0968 - val_acc: 0.5431


## Training

In [38]:
'''
Now that the fully connected layer trained its weights accordingly,
we can start to unfreeze the weights of the network. We will 
only unfreeze top layers. The bottom layers contain very abstract 
feature extractors, which are definitely of use for our data. The top 
layers, however, extract more specific features that were tailored 
for the data its weights were originally trained from (Imagenet).
Therefore, we only want to fine-tune top layer weights, and not bottom 
weights. We chose to fine-tune the top 2 inception modules.
'''
print('Unfreezing top-layer weights...')
for layer in base.layers[:151]:
    layer.trainable = False
for layer in base.layers[151:]:
    layer.trainable = True

'''
After the weights have been unfreezed, the model has to be compiled again.
This time, we will use SGD with a very low learning rate to fine-tune the
model. 
'''
model.compile(optimizer=SGD(lr=1e-4, decay=1e-6, momentum=.9, nesterov=True),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


'''
Train the fully connected layer along with the unfreezed part of
the network to fine-tune the weights to optimize the 
extraction of specific features to our data.
'''
cb = [EarlyStopping(monitor='val_loss', patience=3, verbose=0)]

print('Transferlearning top part of network..')
model.fit(x=X_train, y=y_train, validation_split=0.2, epochs=100, batch_size=50, callbacks=cb)

Unfreezing top-layer weights...
Transferlearning top part of network..
Train on 3063 samples, validate on 766 samples
Epoch 1/100
3063/3063 [==============================] - 752s - loss: 1.1304 - acc: 0.5047 - val_loss: 1.0648 - val_acc: 0.5222
Epoch 2/100
3063/3063 [==============================] - 740s - loss: 1.0980 - acc: 0.5126 - val_loss: 1.0392 - val_acc: 0.5313
Epoch 3/100
3063/3063 [==============================] - 736s - loss: 1.0449 - acc: 0.5475 - val_loss: 1.0064 - val_acc: 0.5522
Epoch 4/100
3063/3063 [==============================] - 743s - loss: 1.0069 - acc: 0.5547 - val_loss: 0.9859 - val_acc: 0.5757
Epoch 5/100
3063/3063 [==============================] - 738s - loss: 0.9683 - acc: 0.5850 - val_loss: 0.9648 - val_acc: 0.5927
Epoch 6/100
3063/3063 [==============================] - 735s - loss: 0.9302 - acc: 0.6046 - val_loss: 0.9495 - val_acc: 0.6084
Epoch 7/100
3063/3063 [==============================] - 735s - loss: 0.8828 - acc: 0.6340 - val_loss: 0.9392 - va

## Save/load model

Saving the model saves both the architecture and weights of the model. Use this after training, so that the model can instantly be used for predictions.

In [39]:
model.save('inception.h5')

In [3]:
from keras.models import load_model
model = load_model('inception.h5')

## Validation


In [40]:
print('Learning done, making predictions..\n')
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

predictions = model.predict(X_test)
y_hat = [np.argmax(p) for p in predictions]
y_tr = [np.argmax(p) for p in y_test]

print('Classification report:')
print(classification_report(y_pred=y_hat, y_true=y_tr))
print('')
print('Confusion matrix:')
print(confusion_matrix(y_pred=y_hat, y_true=y_tr))
print('')
print('Accuracy:')
print(accuracy_score(y_pred=y_hat, y_true=y_tr))

Learning done, making predictions..

Classification report:
             precision    recall  f1-score   support

          0       0.82      0.88      0.85       391
          1       0.71      0.75      0.73       304
          2       0.75      0.57      0.65       159
          3       0.78      0.72      0.75       105

avg / total       0.77      0.77      0.77       959


Confusion matrix:
[[344  43   4   0]
 [ 61 227  13   3]
 [ 12  37  91  19]
 [  4  11  14  76]]

Accuracy:
0.769551616267
